In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
conditions = [
    (train['Pawpularity'] > 80),
    (train['Pawpularity'] <= 80) & (train['Pawpularity'] > 60),
    (train['Pawpularity'] <= 60) & (train['Pawpularity'] > 40),
    (train['Pawpularity'] <= 40) & (train['Pawpularity'] > 20),
    (train['Pawpularity'] <= 20) & (train['Pawpularity'] > 0),
]
values = ['tier_5','tier_4', 'tier_3', 'tier_2', 'tier_1']
train['tier'] = np.select(conditions, values)

In [ ]:
train = train.drop(columns='Id')

In [ ]:
from sklearn.model_selection import train_test_split
X = train[['Eyes','Face','Near','Group','Collage','Occlusion']]
y = train['Pawpularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(cv=3,solver='liblinear').fit(X_train, y_train)
model.score(X_test,y_test)

In [ ]:
from xgboost import XGBRegressor
model1 =  XGBRegressor(
        random_state=42,
        n_estimators=10000,
)
model1.fit(X_train,y_train)

In [ ]:
import lightgbm as lgb
model2 = lgb.LGBMRegressor(num_leaves=50,
                        learning_rate=0.1,
                        n_estimators=10000)
model2.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l2',
        early_stopping_rounds=25)

In [ ]:
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
Test_X = test[['Eyes','Face','Near','Group','Collage','Occlusion']]
prediction1 = model1.predict(Test_X)
prediction2 = model2.predict(Test_X)

In [ ]:
prediction = np.mean(np.array([prediction1,prediction2]),axis=0)

In [ ]:
count = 0
data = []
for i in test.itertuples():
  data.append([i[1],prediction[count]])
  count+=1
data

In [ ]:
prediction_df = pd.DataFrame(data,columns=['Id','Pawpularity'])
prediction_df.to_csv('submission.csv',header=True,index=None)